In [ ]:
import os
import time

os.environ["NEURON_RT_NUM_CORES"] = "2"

from dotenv import load_dotenv
from optimum.neuron import NeuronModelForCausalLM
from optimum.neuron import pipeline
from transformers import AutoTokenizer

load_dotenv()

## Load, Compile, Save

Load a model from the Hugging Face Hub, compile it into Neuron format, and save the compiled model and tokenizer to a local directory.

In [ ]:
# model id you want to compile
vanilla_model_id = "meta-llama/Llama-2-7b-chat-hf"

# configs for compiling model
compiler_args = {"num_cores": 2, "auto_cast_type": "fp16"}
input_shapes = {
  "sequence_length": 2048, # max length to generate
  "batch_size": 1 # batch size for the model
  }

llm = NeuronModelForCausalLM.from_pretrained(vanilla_model_id, export=True, **input_shapes, **compiler_args)
tokenizer = AutoTokenizer.from_pretrained(vanilla_model_id)

In [ ]:
# Save locally or upload to the HuggingFace Hub
out_dir = "mnt/store/"
save_directory = out_dir + "llama_neuron"
llm.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

## Run Inference

Now we can re-load the converted model from the local directory that we saved this in above, and run inference on the provided messages using the `pipeline` function from the `optimum.neuron` library.

In [ ]:
# Load with the HuggingFace Pipeline API
pipe = pipeline("text-generation", save_directory)

In [ ]:
# Create 'messages' and convert into a string the model can understand
messages = [{"role": "user", "content": "Tell me a long story about WW2"}]
inputs = pipe.tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

# Run inference
time_0 = time.time()
outputs = pipe(inputs)
time_1 = time.time()
out_str = outputs[0]["generated_text"][len(inputs):].strip()
tokens = pipe.tokenizer.encode(out_str)

print(out_str[:50], "...", out_str[-50:])
print(f"Total tokens: {len(tokens)}")
print(f"Tokens per second: {len(tokens)/(time_1-time_0)}:.2f")